In [1]:
import requests
import pandas as pd
import json

# Mastodon

## hour_response_mstd

In [10]:
url = f"http://172.26.132.163:5984/mastodon_australia/_design/24hour/_view/response?reduce=true&group=true"

response = requests.get(url, auth=('admin', 'pass'))

if response.status_code == 200:
    data = response.json()
else:
    print("Failed to retrieve view data. Status code:", response.status_code)

json_file = "hour_response_mstd.json"
with open(json_file, 'w') as outfile:
    json.dump(data, outfile)

## hour_sentiment_mstd

In [14]:
url = f"http://172.26.132.163:5984/mastodon_australia/_design/24hour/_view/sentiment?reduce=true&group=true"

response = requests.get(url, auth=('admin', 'pass'))

if response.status_code == 200:
    data = response.json()
else:
    print("Failed to retrieve view data. Status code:", response.status_code)

for row in data['rows']:
    row['value']['average'] = row['value']['average'] * 5
    
json_file = "hour_sentiment_mstd.json"
with open(json_file, 'w') as outfile:
    json.dump(data, outfile)

## weekday_sentiment_mstd

In [11]:
url = f"http://172.26.132.163:5984/mastodon_australia/_design/7daysOfWeek/_view/sentiment?reduce=true&group=true"

response = requests.get(url, auth=('admin', 'pass'))

if response.status_code == 200:
    data = response.json()
else:
    print("Failed to retrieve view data. Status code:", response.status_code)

for row in data['rows']:
    row['value']['average'] = row['value']['average'] * 5

json_file = "weekday_sentiment_mstd.json"
with open(json_file, 'w') as outfile:
    json.dump(data, outfile)

# Twitter

## hour_response

In [13]:
url = f"http://172.26.132.163:5984/loc_twitter_huge/_design/24hour/_view/response?reduce=true&group=true"

response = requests.get(url, auth=('admin', 'pass'))

if response.status_code == 200:
    data = response.json()
else:
    print("Failed to retrieve view data. Status code:", response.status_code)

json_file = "hour_response.json"
with open(json_file, 'w') as outfile:
    json.dump(data, outfile)

## hour_sentiment

In [16]:
url = f"http://172.26.132.163:5984/loc_twitter_huge/_design/24hour/_view/sentiment?reduce=true&group=true"

response = requests.get(url, auth=('admin', 'pass'))

if response.status_code == 200:
    data = response.json()
else:
    print("Failed to retrieve view data. Status code:", response.status_code)
    
json_file = "hour_sentiment.json"
with open(json_file, 'w') as outfile:
    json.dump(data, outfile)

## weekday_sentiment

In [18]:
url = f"http://172.26.132.163:5984/loc_twitter_huge/_design/7daysOfWeek/_view/sentiment?reduce=true&group=true"

response = requests.get(url, auth=('admin', 'pass'))

if response.status_code == 200:
    data = response.json()
else:
    print("Failed to retrieve view data. Status code:", response.status_code)

json_file = "weekday_sentiment.json"
with open(json_file, 'w') as outfile:
    json.dump(data, outfile)

### AFL

In [34]:
url = f"http://172.26.132.163:5984/loc_twitter_huge/_design/aflWords/_view/sentiment?reduce=true&group=true"

response = requests.get(url, auth=('admin', 'pass'))

if response.status_code == 200:
    view_data3 = response.json()
else:
    print("Failed to retrieve view data. Status code:", response.status_code)

In [35]:
data3 = view_data3['rows']

In [43]:
team = [list(d.values())[0][0] for d in data3]
state = [list(d.values())[0][1] for d in data3]
sum = [list(list(d.values())[1].values())[0] for d in data3]
count = [list(list(d.values())[1].values())[1] for d in data3]
avg = [list(list(d.values())[1].values())[2] for d in data3]

In [44]:
data_clean = {
    'team': team,
    'state': state,
    'sum': sum,
    'count': count,
    'avg': avg
}

In [45]:
df = pd.DataFrame(data_clean)
df.head(5)

,team,state,sum,count,avg
0,Adelaide Crows,1,3.303149,130,0.025409
1,Adelaide Crows,2,9.979369,127,0.078578
2,Adelaide Crows,3,4.424787,63,0.070235
3,Adelaide Crows,4,73.350201,1312,0.055907
4,Adelaide Crows,5,1.199574,48,0.024991


In [66]:
grouped = df.groupby('team')
sum_values = grouped['sum'].sum()
count_values = grouped['count'].mean()

In [67]:
df = pd.concat([sum_values, count_values], axis = 1)

In [68]:
df['avg'] = df['sum']/df['count']

In [73]:
df = df.reset_index()

In [74]:
json_data = df.to_json(orient='records')
json_data

'[{"team":"Adelaide Crows","sum":96.4677598329,"count":266.7142857143,"avg":0.361689512},{"team":"Brisbane Lions","sum":53.7727118914,"count":74.1428571429,"avg":0.7252581565},{"team":"Carlton","sum":107.7426000366,"count":232.5714285714,"avg":0.4632667078},{"team":"Collingwood","sum":586.2313349135,"count":760.1428571429,"avg":0.771212055},{"team":"Essendon.","sum":165.0186697585,"count":389.5714285714,"avg":0.4235902781},{"team":"Fremantle","sum":250.6876212865,"count":377.7142857143,"avg":0.6636964255},{"team":"GWS Giants","sum":34.6199587837,"count":66.8333333333,"avg":0.5180043708},{"team":"Geelong Cats","sum":53.5743378766,"count":79.1428571429,"avg":0.676932067},{"team":"Gold Coast Suns","sum":30.8215943528,"count":42.2857142857,"avg":0.7288890556},{"team":"Hawthorn","sum":93.6563558081,"count":159.5714285714,"avg":0.5869243426},{"team":"Melbourne","sum":65.9303526409,"count":84.2857142857,"avg":0.7822245229},{"team":"North Melbourne","sum":46.9775961226,"count":113.7142857143,"